In [137]:
from itertools import combinations
import numpy as np

## Simple Test Problem

In [138]:
CITIES = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]
test_problem = np.load('lab2/test_problem.npy')

## Common tests

In [139]:
problem = np.load('lab2/problem_g_100.npy')

In [140]:
# Negative values?
np.any(problem < 0)

np.False_

In [141]:
# Diagonal is all zero?
np.allclose(np.diag(problem), 0.0)

True

In [142]:
# Symmetric matrix?
np.allclose(problem, problem.T)

True

In [143]:
# Triangular inequality
all(
    problem[x, y] <= problem[x, z] + problem[z, y]
    for x, y, z in list(combinations(range(problem.shape[0]), 3))
)

True

Working...

In [144]:
print(problem)

[[  0.         179.92038688 176.126132   ... 162.61509016 407.15461186
  141.77638964]
 [179.92038688   0.         278.35892376 ... 116.33000682 281.45211088
  154.17643486]
 [176.126132   278.35892376   0.         ... 178.74068495 375.52755063
  312.44633915]
 ...
 [162.61509016 116.33000682 178.74068495 ...   0.         245.06340681
  224.97614312]
 [407.15461186 281.45211088 375.52755063 ... 245.06340681   0.
  434.61707535]
 [141.77638964 154.17643486 312.44633915 ... 224.97614312 434.61707535
    0.        ]]


Greedy algorithm

In [145]:
from typing import List, Tuple
import math

def tsp_greedy_nearest_neighbor(d: List[List[float]], start: int) -> Tuple[List[int], float]:
    """
    Builds a tour using the greedy 'nearest neighbor' strategy starting from a fixed node.
    Returns (tour, cost). The tour includes the return to the start node.
    """
    n = len(d)
    unvisited = set(range(n))
    tour = [start]
    unvisited.remove(start)
    total = 0.0

    current = start
    while unvisited:
        # choose the unvisited node with the minimum distance (tie-break based on the smallest index)
        nxt = min(unvisited, key=lambda j: (d[current][j], j))
        total += d[current][nxt]
        tour.append(nxt)
        unvisited.remove(nxt)
        current = nxt

    # return to the starting node
    total += d[current][start]
    tour.append(start)
    return tour, total


def tsp_greedy_all_starts(d: List[List[float]]) -> Tuple[List[int], float, int]:
    """
    Runs the nearest-neighbor heuristic from every possible starting node and returns the best result.
    Returns (best_tour, best_cost, start_used).
    """
    best_tour, best_cost, best_start = None, math.inf, -1
    for s in range(len(d)):
        tour, cost = tsp_greedy_nearest_neighbor(d, s)
        if cost < best_cost:
            best_tour, best_cost, best_start = tour, cost, s
    return best_tour, best_cost, best_start

In [146]:
# =========================
# Run script
# =========================
tour, cost, start = tsp_greedy_all_starts(problem)
print("Best start:", start)
print("Tour:", tour)
print("Cost:", cost)

Best start: 62
Tour: [62, 29, 76, 96, 98, 39, 65, 59, 9, 82, 23, 61, 95, 38, 35, 17, 44, 94, 79, 45, 73, 66, 84, 3, 51, 28, 21, 60, 11, 2, 58, 70, 56, 18, 72, 31, 48, 10, 42, 64, 97, 30, 81, 68, 52, 83, 89, 85, 77, 6, 12, 91, 15, 50, 24, 40, 19, 80, 25, 26, 13, 90, 16, 53, 57, 41, 14, 0, 33, 54, 67, 32, 37, 5, 49, 7, 99, 27, 20, 74, 71, 47, 63, 93, 46, 36, 4, 75, 34, 55, 86, 87, 43, 92, 88, 78, 8, 1, 69, 22, 62]
Cost: 4464.307330960409


Random solution

In [147]:
import random

def tsp_random_solution(d):
    """
    Generate a random solution for the TSP.
    """
    n = len(d)
    nodes = list(range(n))
    random.shuffle(nodes)
    
    total = 0.0
    for i in range(n - 1):
        total += d[nodes[i]][nodes[i + 1]]
    total += d[nodes[-1]][nodes[0]]
    
    tour = nodes + [nodes[0]]
    return tour, total

In [148]:
# =========================
# Run script
# =========================
tour, cost = tsp_random_solution(problem)
print("Random tour:", tour)
print("Cost:", cost)

Random tour: [56, 49, 94, 72, 74, 80, 13, 14, 37, 30, 86, 48, 20, 63, 67, 73, 83, 10, 29, 55, 34, 39, 75, 77, 54, 44, 90, 58, 91, 6, 4, 53, 89, 52, 3, 8, 84, 26, 12, 92, 23, 95, 97, 88, 32, 43, 40, 17, 16, 98, 85, 42, 78, 57, 41, 11, 61, 51, 82, 66, 28, 25, 35, 21, 45, 62, 99, 69, 2, 81, 22, 65, 18, 96, 31, 71, 5, 1, 38, 60, 47, 68, 64, 7, 0, 9, 33, 79, 46, 70, 24, 93, 59, 36, 50, 27, 87, 15, 19, 76, 56]
Cost: 25386.900156648833


EC solution

In [154]:
import numpy as np
from numba import njit, prange

@njit(parallel=True, fastmath=True)
def population_cost_numba(pop: np.ndarray, D: np.ndarray) -> np.ndarray:
    """
    Computes the cost of the tours in parallel.
    pop: (P, n) population of permutations
    D:   (n, n) distance matrix

    """
    P, n = pop.shape
    out = np.empty(P, dtype=np.float64)
    for i in prange(P):
        s = 0.0
        for k in range(n - 1):
            s += D[pop[i, k], pop[i, k + 1]]
        s += D[pop[i, n - 1], pop[i, 0]]
        out[i] = s
    return out


In [155]:
import numpy as np

RANDOM_SEED = 42

TOURNAMENT_SIZE = 3
ELITISM_RATE = 0.10          # fraction of the best individuals to preserve
PM_SWAP = 0.2                # if None -> 1 / number_of_nodes
MAX_GENERATIONS = 50_000
EARLY_STOP_PATIENCE = 1_000  # maximum number of generations without improvement
POPULATION_SIZE = 300        # fixed population size

# =========================
# Utility
# =========================
def set_seed(seed):
    if seed is not None:
        np.random.seed(seed)

def route_cost(route: np.ndarray, D: np.ndarray) -> float:
    """Cost of a single tour (including the return to the start)."""
    # D[route, np.roll(route, -1)] takes the distance i->i+1 and the last->first
    return D[route, np.roll(route, -1)].sum()

def population_cost(pop: np.ndarray, D: np.ndarray) -> np.ndarray:
    """Cost of all tours in the population (vectorized)."""
    return D[pop, np.roll(pop, -1, axis=1)].sum(axis=1)

def make_initial_population(n_nodes: int) -> np.ndarray:
    """Creates an initial population of size POPULATION_SIZE; each individual is a permutation."""
    pop_size = POPULATION_SIZE
    pop = np.zeros((pop_size, n_nodes), dtype=int)
    base = np.arange(n_nodes)
    for i in range(pop_size):
        pop[i] = np.random.permutation(base)
    return pop

# =========================
# Parent selection
# =========================
def tournament_selection(pop: np.ndarray, fitness: np.ndarray, k: int) -> np.ndarray:
    """Selects one individual via a tournament of k elements (lowest fitness = best)."""
    idxs = np.random.randint(0, len(pop), size=k)
    best = idxs[np.argmin(fitness[idxs])]
    return pop[best].copy()

# # =========================
# # Crossover (Order Crossover - OX)
# # =========================
# def order_crossover(p1: np.ndarray, p2: np.ndarray) -> np.ndarray:
#     n = len(p1)
#     a, b = np.sort(np.random.randint(0, n, size=2))
#     off = -np.ones(n, dtype=int)
#     # copy the segment from p1
#     off[a:b+1] = p1[a:b+1]
#     # fill with the order of p2, skipping duplicates
#     used = set(off[a:b+1])
#     pos = (b + 1) % n
#     for gene in p2:
#         if gene not in used:
#             off[pos] = gene
#             pos = (pos + 1) % n
#     return off


def inver_over(p1: np.ndarray, p2: np.ndarray, pr: float = 0.9) -> np.ndarray:
    """
    Inver-Over crossover (asymmetric) for permutations (TSP on nodes).
    - p1: first parent (the base sequence)
    - p2: second parent (provides the 'edges' to preserve)
    - pr: probability of choosing a random node instead of a neighbor in p2

    Returns a child (permutation) obtained by starting from p1 and applying
    inversions guided by the adjacencies in p2.
    """
    child = p1.copy()
    n = len(child)

    # mapping -> for each node in p2, its two neighbors (cyclic)
    idx2 = {city: i for i, city in enumerate(p2)}
    neigh2 = {city: (p2[(i - 1) % n], p2[(i + 1) % n]) for city, i in idx2.items()}

    # choose an initial gene from the first parent
    c = np.random.choice(child)

    while True:
        # choose the 'target' j: random with probability pr, otherwise a neighbor of c in p2
        if np.random.random() < pr:
            # random node different from c
            cand = child[child != c]
            j = np.random.choice(cand)
        else:
            j = neigh2[c][np.random.randint(0, 2)]  # one of the two neighbors in p2

        # positions of c and j in the current child
        pos_c = int(np.where(child == c)[0][0])
        pos_j = int(np.where(child == j)[0][0])

        # if j is adjacent to c in the child (cyclically), stop
        if ((pos_c - pos_j) % n == 1) or ((pos_j - pos_c) % n == 1):
            break

        # otherwise, invert the segment between c and j (inclusive)
        a, b = sorted((pos_c, pos_j))
        child[a:b+1] = child[a:b+1][::-1]

        # continue from the new 'current' node
        c = j

    return child


# =========================
# Mutations
# =========================

def mutate_swap(route: np.ndarray) -> np.ndarray:
    r = route.copy()
    i, j = np.random.randint(0, len(r), size=2)
    r[i], r[j] = r[j], r[i]
    return r

# =========================
# GA main
# =========================
def genetic_tsp_nodes(distance_matrix: np.ndarray):
    set_seed(RANDOM_SEED)

    n_nodes = distance_matrix.shape[0]
    pop = make_initial_population(n_nodes)

    pm_swap = PM_SWAP
    elite_size = max(1, int(np.round(ELITISM_RATE * len(pop))))

    # inizial evaluation
    fit = population_cost(pop, distance_matrix)
    best_idx = np.argmin(fit)
    best_route = pop[best_idx].copy()
    best_cost = fit[best_idx]
    no_improve = 0

    # print(f"[Init] n_nodes={n_nodes}, pop_size={len(pop)}, elite={elite_size}, "
    #       f"pm_swap={pm_swap:.4f}")
    # print(f"[Init] best_cost={best_cost:.4f}")

    for gen in range(1, MAX_GENERATIONS + 1):
        # ---- Elitism: take the best 'elite_size' individuals
        elite_idxs = np.argsort(fit)[:elite_size]
        elite = pop[elite_idxs].copy()

        # ---- New generation via selection, crossover, and mutation
        new_pop = [elite[i] for i in range(len(elite))]  # preserve the elite

        # fill up to pop_size
        while len(new_pop) < len(pop):
            p1 = tournament_selection(pop, fit, TOURNAMENT_SIZE)
            
            if np.random.random() < 0.2:
                child = mutate_swap(p1)
            else:
                p2 = tournament_selection(pop, fit, TOURNAMENT_SIZE)
                child = inver_over(p1, p2, pr=0.1)

            new_pop.append(child)

        pop = np.array(new_pop, dtype=int)

        # ---- Evaluate
        fit = population_cost_numba(pop, distance_matrix)
        gen_best_idx = np.argmin(fit)
        gen_best_cost = fit[gen_best_idx]

        # ---- Update global best
        if gen_best_cost < best_cost:
            best_cost = gen_best_cost
            best_route = pop[gen_best_idx].copy()
            no_improve = 0
        else:
            no_improve += 1

        # # light logging
        # if gen % 200 == 0 or no_improve == 0:
        #     print(f"[Gen {gen}] best={best_cost:.4f} (gen_best={gen_best_cost:.4f}, no_improve={no_improve})")

        # early stopping
        if no_improve >= EARLY_STOP_PATIENCE:
            print(f"[Stop] No improvement in {EARLY_STOP_PATIENCE} generations.")
            break

    return best_route, best_cost

In [156]:
import os
import numpy as np

# =========================
# Run script on all problems
# =========================

set_seed(RANDOM_SEED)

folder = "lab2"

# List all .npy files in the folder
files = [f for f in os.listdir(folder) if f.endswith(".npy")]

print(f"Found {len(files)} problems in '{folder}':\n", files)

for fname in files:
    path = os.path.join(folder, fname)
    problem = np.load(path)

    assert problem.ndim == 2 and problem.shape[0] == problem.shape[1], "The matrix must be square (n×n)."

    print(f"\n=== Solving {fname} ===")
    best_route, best_cost = genetic_tsp_nodes(problem)

    print("\n=== RESULT ===")
    print("Best tour (nodes in order):", best_route.tolist())
    print("Total cost:", float(best_cost))

Found 22 problems in 'lab2':
 ['problem_g_10.npy', 'problem_g_100.npy', 'problem_g_1000.npy', 'problem_g_20.npy', 'problem_g_200.npy', 'problem_g_50.npy', 'problem_g_500.npy', 'problem_r1_10.npy', 'problem_r1_100.npy', 'problem_r1_1000.npy', 'problem_r1_20.npy', 'problem_r1_200.npy', 'problem_r1_50.npy', 'problem_r1_500.npy', 'problem_r2_10.npy', 'problem_r2_100.npy', 'problem_r2_1000.npy', 'problem_r2_20.npy', 'problem_r2_200.npy', 'problem_r2_50.npy', 'problem_r2_500.npy', 'test_problem.npy']

=== Solving problem_g_10.npy ===
[Stop] No improvement in 1000 generations.

=== RESULT ===
Best tour (nodes in order): [2, 3, 1, 6, 4, 5, 9, 7, 0, 8]
Total cost: 1497.6636482252907

=== Solving problem_g_100.npy ===


KeyboardInterrupt: 